In [0]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import requests
import json
import re

## Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [0]:
path = 'drive/My Drive/marcelo/initiatives/virida.ai/shared/data/'

## Gold Standard dataset

In [0]:
gs_projects = pd.read_pickle(path+'projects/gold_standard_2019-11-17.pkl')
gs_ver = pd.read_pickle(path+'projects/gold_standard_details_2019-12-13.pkl')

In [0]:
ids = pd.unique(gs_ver['project.id'])[:20]
gs_ver[gs_ver['project.id']==ids[2]]

In [5]:
k = gs_projects['id']=='1809'
gs_projects[k]

,carbon_stream,country,country_code,created_at,crediting_period_end_date,crediting_period_start_date,description,estimated_annual_credits,gsf_standards_version,id,latitude,longitude,methodology,name,owner,owner.created_at,owner.id,owner.is_chargeable,owner.number,owner.updated_at,project_developer,size,state,status,sustainable_development_goals,sustaincert_id,sustaincert_url,type,updated_at
15,Reporting Only,Kenya,KE,2019-03-27T13:54:47Z,2018-06-07,2018-06-06,The sustainable agroforestry based dairy value...,100000,0.9,1809,1.1,34.3,Other,Sustainable Agroforestry Based Dairy Value Cha...,NaN,2019-03-27T11:23:57Z,979,False,1043294,2019-03-27T11:23:57Z,Livelihoods Fund SICAV SIF,Microscale,None,GOLD_STANDARD_CERTIFIED_DESIGN,[{'name': 'Goal 12: Responsible Production and...,6588,https://impact.sustain-cert.com/public_project...,Other,2019-08-26T19:09:59Z


#### Extract vintages and issuances from GS data

In [0]:
# Take one row of issuances dataframe and splits into ranges within each vintage/year
def extract_vintages_gs(issuance,date_format):

  # Convert dates from string
  date_begin = pd.to_datetime(issuance['monitoring_period_start_date'],format=date_format)
  date_end = pd.to_datetime(issuance['monitoring_period_end_date'],format=date_format)

  # Extract date range for vintage
  # Take into account begin and end of monitoring period for pro-rated issuances
  df = {'vintage':[issuance['vintage']],
        'begin':[max(datetime(int(issuance['vintage']),1,1),date_begin)],
        'end':[min(datetime(int(issuance['vintage']),12,31),date_end)],
        'monitoring_begin':date_begin,
        'monitoring_end':date_end,
        'issued':[issuance['number_of_credits']]}

  return pd.DataFrame(df)

In [0]:
def extract_issuances_gs(ids,issuances,projects,date_format='%Y-%m-%d'):
# Extract all issuances from GS raw data

  # Run issuances extraction
  w = []
  for i in ids:
    # Subset by project, only VERs that were issued (not retired, duplicated in raw date)
    k = issuances['project.id']==i
    y = issuances[k].groupby(['monitoring_period_start_date',
                              'monitoring_period_end_date',
                              'vintage'])['number_of_credits'].sum().to_frame().reset_index()

    # Compute date ranges for vintages, given multi-year monitoring period
    x = []
    for j in range(len(y)):
      x = x+[extract_vintages_gs(y.loc[j,:],date_format=date_format)]
    y = pd.concat(x).reset_index(drop=True)

    # Consolidate date ranges by vintage
    # Add a period with zero issuance at the crediting_period_start_date
    # Make the begin date the minimum of all dates for a certain vintage 
    # Make the end date the maximum of all dates for a certain vintage
    y = pd.concat([y.groupby(['vintage'])['issued'].sum().to_frame().reset_index(),
                   y.groupby(['vintage'])['begin'].min().reset_index()['begin'],
                   y.groupby(['vintage'])['end'].max().reset_index()['end'],
                   y.groupby(['vintage'])['monitoring_begin'].min().reset_index()['monitoring_begin'],
                   y.groupby(['vintage'])['monitoring_end'].max().reset_index()['monitoring_end']],axis=1)

    # Complete date gaps in monitoring with zero issuance
    b = y['begin'].shift(periods=-1).dropna().reset_index(drop=True)-np.timedelta64(1,'D')
    if (len(b)>0):
      a = y['end'].shift(periods=1).dropna().reset_index(drop=True)
      x = b-a
      k = x>np.timedelta64(0,'D')
      if (sum(k)>0):
        c = {'begin':a[k].reset_index(drop=True)+np.timedelta64(1,'D'),
             'end':b[k].reset_index(drop=True),
             'issued':[0]*sum(k),
             'monitoring_begin':y['monitoring_begin'][k],
             'monitoring_end':y['monitoring_end'][k]}
        y = pd.concat([y,pd.DataFrame(c)]).reset_index(drop=True)

    # Add project id, pro-rated annual estimated emissions
    # Compute issuance rate and issuance count
    # Add crediting period dates
    y['id'] = i
    a = projects[projects['id']==i]['estimated_annual_credits'].values[0]
    y['annual'] = a*(y['end']-y['begin']).dt.days.astype('int')/365
    y['rate'] = y['issued']/y['annual']
    y = y.reset_index()
    y['count'] = y.index+1
    k = projects['id']==i
    y['crediting_begin'] = pd.to_datetime(projects[k]['crediting_period_start_date']
                                          .values[0],format=date_format)
    y['crediting_end'] = pd.to_datetime(projects[k]['crediting_period_end_date']
                                          .values[0],format=date_format)

    w = w+[y[['id','vintage','issued','annual','rate','count','begin','end',
              'monitoring_begin','monitoring_end','crediting_begin','crediting_end']]]

  return pd.concat(w).reset_index(drop=True)

In [0]:
# Filter out projects that have not issued VERs
k = (gs_ver['product.abbreviation']=='VER')&(gs_ver['status']=='ISSUED')
ver = gs_ver[k]
ids = pd.unique(ver['project.id'])[:5]
#ids = ['1806']
extract_issuances_gs(ids=ids,issuances=ver,projects=gs_projects)

#### Get SDGs from GS data

In [14]:
k = gs_projects['id']=='1806'
gs_projects[k]['sustainable_development_goals'].values[0]

[{'issuable_products': [], 'name': 'Goal 15: Life On Land'},
 {'issuable_products': [],
  'name': 'Goal 12: Responsible Production and Consumption'},
 {'issuable_products': [{'abbreviation': 'VER',
    'name': 'Verified Emission Reduction'},
   {'abbreviation': 'CER', 'name': 'Certified Emission Reduction'},
   {'abbreviation': 'REC', 'name': 'Renewable Energy Certificate'},
   {'abbreviation': 'PER', 'name': 'Planned Emission Reduction'}],
  'name': 'Goal 13: Climate Action'}]

In [23]:
sdg = gs_projects[k]['sustainable_development_goals'].values[0]
[re.search(' (.+?):',x['name']).group(1) for x in sdg]

['15', '12', '13']

## Verra dataset

In [0]:
verra_vcu = pd.read_pickle(path+'projects/verra_vcu_main_2019-11-17.pkl')
verra_projects = pd.read_pickle(path+'projects/verra_projects_2019-11-17.pkl')
verra_details = pd.read_pickle(path+'projects/verra_details_2019-11-17.pkl')

#### Extract vintages and issuances from Verra data

In [0]:
def extract_vintages_verra(issuance,date_format):
# Extract vintages given a monitoring date range, which may be multi-year

  # Compute vintage date ranges given monitoring range
  y = pd.unique(pd.to_datetime(pd.date_range(start=issuance['vintagePeriodStart'][0],
                                             end=issuance['vintagePeriodEnd'][0]-
                                             timedelta(days=1),freq='D')).year)
  d = [[datetime(x,1,1),datetime(x,12,31)] for x in y]
  
  # Split dataframe according to vintage date ranges
  issuance['range'] = [d]
  issuance = issuance.explode('range')
  issuance = pd.concat([issuance,issuance['range'].apply(pd.Series).reindex()],axis=1)
  issuance.columns = ['monitoring_begin','monitoring_end',
                      'issued','annual','range','begin','end']
  issuance['begin'] = issuance[['monitoring_begin','begin']].max(axis=1)
  issuance['end'] = issuance[['monitoring_end','end']].min(axis=1)

  # Compute issuance share in date range
  issuance['share'] = (issuance['end']-issuance['begin']).dt.days.astype('int')/(
      issuance['monitoring_end']-issuance['monitoring_begin']).dt.days.astype('int')
  issuance['issued'] = issuance['issued']*issuance['share']

  # Extract vintage
  issuance['vintage'] = pd.DatetimeIndex(issuance['end']).year

  # Adjust annual estimated by date range
  issuance['annual'] = issuance['annual']*(issuance['end']-issuance['begin']).dt.days.astype('int')/365
  #issuance = issuance[['vintage','begin','end','issued','annual']]

  return issuance[['vintage','begin','end','issued','annual','monitoring_begin','monitoring_end']]

In [0]:
def extract_issuances_verra(ids,issuances,projects,date_format='%Y-%m-%d'):
# Extract all issuances by project, vintage, issued amount, annual estimated, issuance rate and issuance count

  # Convert dates from string
  issuances['vintagePeriodStart'] = pd.to_datetime(issuances['vintagePeriodStart'],
                                                   format=date_format)
  issuances['vintagePeriodEnd'] = pd.to_datetime(issuances['vintagePeriodEnd'],
                                                 format=date_format)
  
  # Run issuances extraction
  w = []
  for i in ids:
    # Subset by project
    k = issuances['projectId']==i
    y = issuances[k].groupby(['vintagePeriodStart',
                              'vintagePeriodEnd']
                             )['quantityToIssue'].sum().to_frame().reset_index()
    
    # Complete date gaps in monitoring with zero issuance 
    b = y['vintagePeriodStart'].shift(periods=-1).dropna().reset_index(drop=True)-np.timedelta64(1,'D')
    if (len(b)>0):
      a = y['vintagePeriodEnd'].shift(periods=1).dropna().reset_index(drop=True)
      x = b-a
      k = x>np.timedelta64(0,'D')
      if (sum(k)>0):
        c = {'vintagePeriodStart':a[k].reset_index(drop=True)+np.timedelta64(1,'D'),
             'vintagePeriodEnd':b[k].reset_index(drop=True),
             'quantityToIssue':[0]*sum(k)}
        y = pd.concat([y,pd.DataFrame(c)]).reset_index(drop=True)

    # Add annual estimated emission reductions to dataframe
    k = projects['projectId']==i
    y['annual'] = projects[k]['estimatedAnnualEmissionReductions'].values[0]

    # Extract date ranges, issuances and annual estimated by vintage
    x = []
    for j in range(len(y)):
      v = y[j:(j+1)]
      v = v.reset_index(drop=True)
      x = x+[extract_vintages_verra(v,date_format=date_format)]
    y = pd.concat(x).reset_index(drop=True)
    y = pd.concat([y.groupby(['vintage']).sum()[['issued','annual']].reset_index(),
                   y.groupby(['vintage'])['begin'].min().reset_index()['begin'],
                   y.groupby(['vintage'])['end'].max().reset_index()['end'],
                   y.groupby(['vintage'])['monitoring_begin'].min().reset_index()['monitoring_begin'],
                   y.groupby(['vintage'])['monitoring_end'].max().reset_index()['monitoring_end']],axis=1)

    # Compute issuance rate,  issuance count and add project ID
    y['rate'] = y['issued']/y['annual']
    y = y.reset_index()
    y['count'] = y.index+1
    y['id'] = i

    w = w+[y[['id','vintage','issued','annual','rate','count','begin','end','monitoring_begin','monitoring_end']]]

  return pd.concat(w).reset_index(drop=True)

In [0]:
ids = pd.unique(verra_vcu['projectId'])[0:10]
#ids = ['1094']
extract_issuances_verra(ids=ids,issuances=verra_vcu,projects=verra_projects)

In [0]:
verra_vcu[0:5]

## General dataset

#### Map countries to regions and subregions

In [0]:
def get_countries(iso_list):

  country = {}
  region = {}
  subregion = {}

  for iso in iso_list:
    try:
      if (iso not in country) or (iso not in region) or (iso not in subregion):
        response = json.loads(requests.get(f'https://restcountries.eu/rest/v2/alpha/{iso}').text)
        country[iso] = response['name']
        region[iso] = response['region']
        subregion[iso] = response['subregion']
    except:
        print(f"Couldn't retrieve data for {iso}")

  return [country,region,subregion]

In [9]:
get_countries(iso_list=gs_projects['country_code'].tolist())

[{'AD': 'Andorra',
  'AE': 'United Arab Emirates',
  'AF': 'Afghanistan',
  'AR': 'Argentina',
  'AU': 'Australia',
  'AW': 'Aruba',
  'AX': 'Åland Islands',
  'BD': 'Bangladesh',
  'BF': 'Burkina Faso',
  'BG': 'Bulgaria',
  'BI': 'Burundi',
  'BJ': 'Benin',
  'BO': 'Bolivia (Plurinational State of)',
  'BR': 'Brazil',
  'CA': 'Canada',
  'CD': 'Congo (Democratic Republic of the)',
  'CG': 'Congo',
  'CH': 'Switzerland',
  'CI': "Côte d'Ivoire",
  'CL': 'Chile',
  'CM': 'Cameroon',
  'CN': 'China',
  'CO': 'Colombia',
  'CR': 'Costa Rica',
  'DJ': 'Djibouti',
  'DO': 'Dominican Republic',
  'DZ': 'Algeria',
  'EG': 'Egypt',
  'ER': 'Eritrea',
  'ET': 'Ethiopia',
  'GE': 'Georgia',
  'GH': 'Ghana',
  'GN': 'Guinea',
  'GT': 'Guatemala',
  'HN': 'Honduras',
  'HT': 'Haiti',
  'HU': 'Hungary',
  'ID': 'Indonesia',
  'IN': 'India',
  'IQ': 'Iraq',
  'KE': 'Kenya',
  'KH': 'Cambodia',
  'KM': 'Comoros',
  'KZ': 'Kazakhstan',
  'LA': "Lao People's Democratic Republic",
  'LK': 'Sri Lanka',
